<font size='4'><b>Color Classification</b></font>

In this project we want to create a Deep learning model for car color classification. Along the project we will first prepare the data for training and testing, then we will fit our data into a convolutional network model, and lastly we will feed our trained model new images and predict their classes.

We import the important modules for our data preparations. We use numpy for matrix and array operations, Matplotlib and cv2 for image manipulations and us will use OS for some file pathing interaction.

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import cv2
import os

<font size='3'><b>1. Data preparation</b></font>

Before training our model we need to prepare the data. The data are images of different cars classed by their color, and placed in different folders in function of their colors. There are 5 different classes of color (5 folders), white, red, green, blue and black. And each class (folder) contains 640 images of cars. (Exception: Green, 420 images). In total there are 2980 image samples available for training.

The different class folders are placed inside DATADIR file "DATASET"

In [ ]:
DATADIR = "DATASET"
CATEDIR = ["White","Red","Green","Blue","Black"]

In order to access the image inside a class, we use 2 'for' loops. The first loop gives access to the DATADIR, and each class folder CATEDIR. The second loop gives access to the images inside the class folders.
The image in itself is useless as a data, we use cv2.imread in order to translate the image into an array. And with matplotlib imshow we can have a display of the image.

In [ ]:
for class_folder in CATEDIR:
# access to the inside DATASET folder containing the different class folders
    path = DATADIR+"/"+class_folder
    for img in os.listdir(path):
    # access to the inside of each class folder, where the images samples are
        img_path = path+"/"+img
        # transform image into an array
        img_array = cv2.imread(img_path)
        plt.imshow(cv2.cvtColor(img_array, cv2.COLOR_BGR2RGB))
        plt.show
        break
    break

The image as an array has 3 parameters, 2 parameters for the size, the weight and the height, and one parameter the number of color layers (3, in our case, referencing to RGB)

In [ ]:
print("image shape: ", img_array.shape)

In [ ]:
print("image array: ", img_array)

In certain cases the images from our dataset can come with different size. In order to have a dataset uniform, we will resize the images using cv2.resize.

In [ ]:
IMG_SIZE = 100

new_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE), 3)
plt.imshow(cv2.cvtColor(new_array, cv2.COLOR_BGR2RGB))
plt.show

The resized image retains all its features, and the image is still recognizable.
The shape of the new image array also retains the 3 parameters, but with different values for the 1st and 2nd parameters, corresponding to the weight and height.

In [ ]:
print("new image shape: ", new_array.shape)

In [ ]:
print("new image array: ", new_array)

<font size='3'><b>2. Creating the training data</b></font>

We have now all the necessary tools, to create the training data for our neural network model. Same as previously, we use the double 'for' loops in order to access and retrieve the image samples inside each class folder, and we want to assign indexes to each of our samples data folders.

We also transform each image into an array, resize them, and save them alongside their class index, inside training_data.

In [ ]:
training_data = []
IMG_SIZE = 100

def create_training_data():
    for class_folder in CATEDIR:
        path = DATADIR+"/"+class_folder
        # give an index to each class folder
        class_index = CATEDIR.index(class_folder)
        for img in os.listdir(path):
            img_array = cv2.imread(path+"/"+img)
            new_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE), 3)
            training_data.append([new_array, class_index])
create_training_data()

We can observe the length of training_data corresponding to the total number of image samples.

In [ ]:
print("number of image samples: ", len(training_data))

By displaying some of the samples in our training data, we observe how our data samples are arranged.

In [ ]:
for sample in training_data[:10]:
    print(sample[1])

In [ ]:
# separating the images from the classes inside training_data
X = []
y = []

for images, classes in training_data:
    X.append(images)
    y.append(classes)
    
X = np.array(X).reshape(-1, IMG_SIZE, IMG_SIZE, 3)
y = np.array(y)

We can see that the image sample are ordered by their classes. All the images of white car first, followed by the red one, and so on.

In [ ]:
# displaying the first 10 image samples of our training data
fig = plt.figure(figsize=(IMG_SIZE, IMG_SIZE))

for i,img in enumerate(X[:10]):
    fig.add_subplot(4,5, i+1)
    image = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    plt.imshow(image)
plt.show()

In order to be more efficient in the training process, we will shuffle our image samples inside training_data, by importing the module random.
With random.shuffle we can shuffle the order of the image samples inside training data randomly.

In [ ]:
import random
random.shuffle(training_data)

for sample in training_data[:10]:
    print(sample[1])

In [ ]:
# separating the images from the classes inside training_data
X = []
y = []

for images, classes in training_data:
    X.append(images)
    y.append(classes)
    
X = np.array(X).reshape(-1, IMG_SIZE, IMG_SIZE, 3)
y = np.array(y)

We can see that the image sample are ordered randomly inside training_data.

In [ ]:
fig = plt.figure(figsize=(IMG_SIZE, IMG_SIZE))

for i,img in enumerate(X[:10]):
    fig.add_subplot(4,5, i+1)
    image = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    plt.imshow(image)
plt.show()

After separating the images and the indexes, from the training_data, we can observe the both of them have the same length, corresponding to the number of samples.

In [ ]:
print("input shape: ", X.shape)

In [ ]:
print("output shape: ", y.shape)

<font size='3'><b>3. Creating the training model</b></font>

We now have the training data needed to train a neural network model with. Next we need to create the neural network, and we choose to use keras and tensorflow. We import the different keras modules needed for creating a neural network model: Sequential to define the type of model; Dense, Conv2D refers respectively to a deep neural network and a convolutional network; MaxPooling2D is used by the convolutional network; and Flatten is used in order to transform a 2D convolutional network into a fully connected network.

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten
from tensorflow.keras.utils import to_categorical

We have our images, data samples in X and their classes in y. In order to understand how well our model is doing with different samples not used for training, we split the data into training data and validation data, with the use of train_test_split from sklearn. We split the data into 70% training data and 30% testing data.

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3,
                                                   shuffle = True) 

In [ ]:
X_train = X_train / 255.0
X_test = X_test / 255.0

In [ ]:
print("training model input shape: ", X_train.shape, "\n")
print("testing model input shape: ", X_test.shape)

In [ ]:
print("training model input: ", X_train)

In [ ]:
y_train = to_categorical(y_train, len(CATEDIR))
y_test = to_categorical(y_test, len(CATEDIR))

In [ ]:
print("training model output shape: ", y_train.shape, "\n")
print("testing model output shape: ", y_test.shape)

In [ ]:
print("training model output: ", y_train)

Now, we want to create our training model, we decided to create a convolutional neural network model which is more adapted for image recognition. <br>
First, we add the input layer with the input_shape corresponding to the shape of the images, and we specify the layer activation rule. <br>
We add more convolutional layers with the activation rule, before flattening the model into a fully connected dense layer, and finally adding the output layer.<br>
We have a classification problem, the activation softmax for the output layer is more adequate.

In [ ]:
model = Sequential()

model.add(Conv2D(16, (3, 3), input_shape = (IMG_SIZE, IMG_SIZE, 3), 
                 activation = "relu"))
model.add(MaxPooling2D(pool_size = (2, 2)))

model.add(Conv2D(32, (3, 3), activation = "relu"))
model.add(MaxPooling2D(pool_size = (2, 2)))

model.add(Conv2D(32, (3, 3), activation = "relu"))
model.add(MaxPooling2D(pool_size = (2, 2)))

model.add(Flatten())
model.add(Dense(64, activation = "relu"))

model.add(Dense(len(CATEDIR), activation = "softmax"))

After adding the layers we compile them into our model, and using the Adam algorithm as the optimizer. As a classification problem, the loss function of the model can be obtained with the categorical_crossentropy.

In [ ]:
model.compile(loss = "categorical_crossentropy", 
              optimizer = "adam", 
              metrics = ["accuracy"])

Now our training model is created and can be used to train with the different image samples, we can see the structure of our model using the model.summary.

In [ ]:
model.summary()

We fit the training data, which mean training our model with the image sample. And we specify the batch_size, corresponding to how many samples at a time are trained, then we specify the number of epochs, which is how much time the model are going to run throughout the sample, and we specify the validation data, to understand how our model fare with image different from the training image samples.

In [ ]:
history = model.fit(X_train, y_train, batch_size = 64, epochs = 10, 
          validation_data = (X_test, y_test))

<b>Small conclusion:</b><br>
We can observe that our model is relatively doing well, despite the number of samples, we obtain around 90% accuracy for the training, and 85% - 87% for the validation. In order to have a better understanding of the result, we plot the evolution of the accuracy and loss of the training and validation, in function of the number of epochs.

In [ ]:
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

With the graphics, we may observe some indication of overfitting which can be explained by the simplicity of our model, and our technique of training. 

In [ ]:
model.save("model_v1_1.h5")

<font size='3'><b>4. Image prediction</b></font>

We have created a model capable of predicting color of a car, and we want to test the model using completely different images from the training or validation data. Those images aren't arranged inside the class folder so they don't posses a class label, we want to use our model to classify them (meaning determining their color for us).

In [ ]:
PREDDIR = "PREDICT"
CATEDIR = ["White","Red","Green","Blue","Black"]

In [ ]:
prediction_data = []
IMG_SIZE = 100

for img in os.listdir(PREDDIR):
    img_array = cv2.imread(PREDDIR+"/"+img)
    new_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE), 3)
    prediction_data.append(new_array)

In [ ]:
X_pred = np.array(prediction_data).reshape(-1, IMG_SIZE, IMG_SIZE, 3)

We reload our saved model

In [ ]:
from tensorflow.keras.models import load_model

In [ ]:
model = load_model("model_v1_1.h5", compile = True)
model.summary()

We predict the classes of the new input images.

In [ ]:
y_pred = model.predict(X_pred)

In order to see the accuracy of the model, bellow are the images that we want to be classified.

In [ ]:
fig = plt.figure(figsize=(IMG_SIZE, IMG_SIZE))

for i,img in enumerate(X_pred):
    fig.add_subplot(4,5, i+1)
    image = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    plt.imshow(image)
plt.show()

We have bellow, the prediction from our model

In [ ]:
for i in range(10):
    result = y_pred[i,:]
    print("image", i)
    for j in range(5):
        if result[j] == 1.0:
            print("color: ", CATEDIR[j], "\n")

<b>Small conclusion:</b><br>
We can observe our model is capable to classify the color of a car, which was our objective in this project. <br>
The prediction of our model isn't 100% accurate, but the result can be perfected by tunning our neural network model and tweaking with the training technique.

<font size='3'><b>5. Different training model / Training technique</b></font>